In [3]:
from CNN_musical.CNN_musical import ModeloCNN, fit
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [7]:
from utils import utils

In [ ]:
import CNN_musical

In [2]:
import os
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# data_dir = "../data/spectrograms"
data_dir = "data/MelSpectrograms/"

#load the train and test data
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [3]:
#No ejectutar si vamos a cargar el modelo
num_epochs = 7
opt_func = torch.optim.Adam
lr = 0.001

model = ModeloCNN(device)
#fitting the model on training data and record the result after each epoch

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [4]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 32
val_size = 580
test_size = 300
train_size = len(dataset) - (val_size + test_size)

train_data,val_data,test_data = random_split(dataset,[train_size,val_size,test_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")
print(f"Length of Test Data : {len(test_data)}")


#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)
test_dl = DataLoader(test_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)

Length of Train Data : 3216
Length of Validation Data : 580
Length of Test Data : 300


In [10]:
utils.separar("./datos_nuevos/Guitarra1.wav", "datos_nuevos/prueba/", 2)

NameError: name 'librosa' is not defined

# Cargar el modelo y predecir

In [6]:
def cargarModelo(ruta, device):
    model = ModeloCNN(device)
    model.load_state_dict(torch.load(ruta, map_location=device))
    return model

In [7]:
ruta = "../trainedModelNoHistory.pth"
device = torch.device('cpu')

model = cargarModelo(ruta, device)

In [9]:
def single_predict(model, input):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        predicted_index = predictions[0].argmax(0)
    return predicted_index.item()

In [14]:
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder

def predict(model, data_dir, device):
    dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
    ]))
    data=DataLoader(dataset, 1, num_workers = 4, pin_memory = True)

    predicciones = []
    for batch in data:
        indicePredicho=single_predict(model,batch[0].to(device))
        predicciones.append(indicePredicho)
    
    return predicciones

In [15]:
def most_common(predicciones):
    return max(set(predicciones), key=predicciones.count)

In [18]:
instrumentos = ['bass', 'brass', 'flute', 'guitar', 'keyboard', 'mallet', 'organ', 'reed', 'string', 'vocal']

ruta = "./pruebas/"

predicciones = predict(model, ruta, device)
prediccionFinal = most_common(predicciones)

print(instrumentos[prediccionFinal])

bass
